## Hottbox CMTF on Synthetic Data

#### Measuring speed and accuracy of Python's Hottbox CMTF on Synthetic Data

#### Loading Data from MATLAB to Python

In [51]:
# Importing MATLAB data to Python
import numpy as np
import tensorly as tl
import functools
import warnings
import scipy.io as sio

# Ranks used
ranks = sio.loadmat('MAT_ranks5.mat')['rankArr']

# Tensors and Matrices from MATLAB (stored in arrays)
Tcont = sio.loadmat('MAT_Tcont5.mat')['Tcont']
# Tcont: 4 (sizes) x 8 (ranks) x 5 (trials) x 10x10x10 (tensor)
Mcont = sio.loadmat('MAT_Mcont5.mat')['Mcont']
# Mcont: 4 (sizes) x 8 (ranks) x 5 (trials) x 10x10 (matrix)

In [86]:
# Tcont[0][x][0][y][0][z].shape
# x = size
# y = rank
# z = trial

#### Run Python CMTF & store 4 measurement data

In [79]:
# Run Py_cmtf over all sizes, ranks, trials

Py_wallTime = np.empty([4, 8, 6])
Py_cpuTime = np.empty([4, 8, 6])
Py_Trecon = np.empty([4, 8, 6])
Py_Mrecon = np.empty([4, 8, 6])

for sz in range(4):
    for r in range(8):
        for itr in range(5):
            wallTime, cpuTime, Trecon, Mrecon = Py_cmtf(Tcont[0][sz][0][r][0][itr], 
                                                        Mcont[0][sz][0][r][0][itr], ranks[0][r], sz)
            Py_wallTime[sz][r][itr] = wallTime
            Py_cpuTime[sz][r][itr] = cpuTime
            Py_Trecon[sz][r][itr] = Trecon
            Py_Mrecon[sz][r][itr] = Mrecon
            
        Py_wallTime[sz][r][5] = np.average(Py_wallTime[sz][r][0:5])
        Py_cpuTime[sz][r][5] = np.average(Py_cpuTime[sz][r][0:5])
        Py_Trecon[sz][r][5] = np.average(Py_Trecon[sz][r][0:5])
        Py_Mrecon[sz][r][5] = np.average(Py_Mrecon[sz][r][0:5])

In [85]:
# Saving 4 measurements to files
np.save('Py_wallTime', Py_wallTime)
np.save('Py_cpuTime', Py_cpuTime)
np.save('Py_Trecon', Py_Trecon)
np.save('Py_Mrecon', Py_Mrecon)

In [91]:
# newfile = np.load('Py_wallTime.npy')
Py_wallTime[0][7][5]

0.08412365913391114

In [68]:
# Main function to run Python's Hottbox CMTF on a tensor, matrix given rank

import scipy.io as sio
import time
    
def Py_cmtf(tensor, matrix, rank, sz):
    """Run Hottbox CMTF using tensor and matrix from MATLAB data.
    Parameters:
        tensor: symmetric tensor to decompose
        matrix: symmetric matrix to decompose
        rank: rank
        sz: size index used for tensor/matrix from [10, 25, 50, 75]
        
    Returns:
        wallTime: Python wall times
        cpuTime: Python CPU times
        Trecon: Python tensor recon error
        Mrecon: Python matrix recon error
    """
    
    sizeI = [10, 25, 50, 75]
    size = sizeI[sz]
    
    # Initialize prelim variables
    CMTFobj = CMTF2(100)

    T = Tensor(tensor)
    M = Tensor(matrix)

    # extra 2 matrices are all ones for Hottbox to work
    M2 = Tensor(np.ones([size, size]))
    M3 = Tensor(np.ones([size, size]))

    # Start Time (Wall Time)
    stw_t1 = time.time()

    # Start Time (CPU Time)
    stc_t1 = time.process_time()

    # Run CMTF
    fmat_a, fmat_b, t_recon, m_recon = CMTFobj.decompose(T, [M, M2, M3], rank)

    # End time (Wall Time)
    etw_t1 = time.time()

    # End time (CPU Time)
    etc_t1 = time.process_time()

    # Elapsed time (Wall Time)
    elapsed_tw_t1 = etw_t1 - stw_t1

    # Elapsed time (CPU Time)
    elapsed_tc_t1 = etc_t1 - stc_t1

    ## Tensor, Matrix reconstruction differences
    Tdiff = Tensor(T[:,:,:] - t_recon[:,:,:])
    Mdiff = Tensor(M[:,:] - m_recon[0][:,:])

    ## Save Python measurements
    wallTime = elapsed_tw_t1
    cpuTime = elapsed_tc_t1
    Trecon = Tdiff.frob_norm
    Mrecon = Mdiff.frob_norm
    
    return wallTime, cpuTime, Trecon, Mrecon

#### Hottbox CMTF code (edited)


In [13]:
import warnings
import numpy as np
from hottbox.algorithms.decomposition.cpd import BaseCPD
from hottbox.core.structures import Tensor
from hottbox.core.operations import khatri_rao, hadamard
from hottbox.utils.generation.basic import super_diag_tensor

# added
import tensorly as tl
import functools

# TODO: Organise this better - lazy work around used
class CMTF2(BaseCPD):
    """ Coupled Matrix and Tensor factorization for two ``Tensors`` of order n and 2 with respect to a specified `rank`.

    Computed via alternating least squares (ALS)

    Parameters
    ----------
    max_iter : int
        Maximum number of iteration
    epsilon : float
        Threshold for the relative error of approximation.
    tol : float
        Threshold for convergence of factor matrices
    random_state : int
    verbose : bool
        If True, enable verbose output
    
    Attributes
    ----------
    cost : list
        A list of relative approximation errors at each iteration of the algorithm.

    References
    ----------
    -   Acar, Evrim, Evangelos E. Papalexakis, Gozde Gurdeniz, Morten A. Rasmussen,
        Anders J. Lawaetz, Mathias Nilsson and Rasmus Bro.
        “Structure-revealing data fusion.” BMC Bioinformatics (2013).
    -   Jeon, Byungsoo & Jeon, Inah & Sael, Lee & Kang, U. (2016).
        SCouT: Scalable coupled matrix-tensor factorization—Algorithm and discoveries.
        Int. Conf. Data Eng.. 811-822. 10.1109/ICDE.2016.7498292.
    """
    # TODO: change init use requiring a change in TensorCPD
    def __init__(self, max_iter=100, epsilon=10e-8, tol=10e-8,
                 random_state=None, verbose=False) -> None:
        super(CMTF2, self).__init__(init='random',
                                   max_iter=max_iter,
                                   epsilon=epsilon,
                                   tol=tol,
                                   random_state=random_state,
                                   verbose=verbose)
        self.cost = []

    def copy(self):
        """ Copy of the CPD algorithm as a new object """
        new_object = super(CMTF2, self).copy()
        new_object.cost = []
        return new_object

    @property
    def name(self):
        """ Name of the decomposition

        Returns
        -------
        decomposition_name : str
        """
        decomposition_name = super(CMTF2, self).name
        return decomposition_name

    def decompose(self, tensor, mlst, rank):
        """ Performs factorisation using ALS on the two instances of ``tensor``
            with respect to the specified ``rank``

        Parameters
        ----------
        tensor : Tensor
            Multi-dimensional data to be decomposed
        mlst : List of `Tensor`
            List of two-dimensional `Tensor` to be decomposed
        rank : tuple # changed to int
            Desired Kruskal rank for the given ``tensor``. Should contain only one value.
            If it is greater then any of dimensions then random initialisation is used
        
        Returns
        -------
        (fmat_a, fmat_b, t_recon, m_recon) : List(np.ndarray) or np.ndarray
            fmat_a, fmat_b are the list of components obtained by applying CMTF
            t_recon, m_recon : The reconstructed tensor and list of matrices

        """
        if not isinstance(tensor, Tensor):
            raise TypeError("Parameter `tensor` should be `Tensor`!")
        if not isinstance(mlst, list):
            raise TypeError("Parameter `mlst` should be a list of `Tensor`!")
        #if not isinstance(rank, tuple):
            #raise TypeError("Parameter `rank` should be passed as a tuple!")
        #if len(rank) != 1:
            #raise ValueError("Parameter `rank` should be tuple with only one value!")
        if not all(isinstance(m, Tensor) for m in mlst):
            raise TypeError("Parameter `mlst` should be a list of `Tensor`!")
        if not all(m.order == 2 for m in mlst):
            raise ValueError("All elements of `mlst` should be of order 2. It is a list of matrices!")

        modes = np.array([list(m.shape) for m in mlst])
        num_modes = len(modes)

        fmat_a, fmat_b = self._init_fmat(modes[:, 0], modes[:, 1], rank)
        
        norm = tensor.frob_norm
        
        for n_iter in range(self.max_iter):
            # Update tensor factors
            for i in range(num_modes):
                _v = hadamard([np.dot(a_i.T, a_i) for k, a_i in enumerate(fmat_a) if k != i])
                _v += fmat_b[i].T.dot(fmat_b[i])
                # print(_v) # output: 3x3 matrix wth values
                
                kr_result = khatri_rao(fmat_a, skip_matrix=i, reverse=True)
                
                _prod_a = np.concatenate([tensor.unfold(i, inplace=False).data, mlst[i].data], axis=1)
                _prod_b = np.concatenate([kr_result.T, fmat_b[i].T], axis=1).T
                
                
                fmat_a[i] = _prod_a.dot(_prod_b).dot(np.linalg.pinv(_v))
            for i in range(num_modes):
                fmat_b[i] = mlst[i].data.T.dot(np.linalg.pinv(fmat_a[i]).T)

            t_recon, m_recon = self._reconstruct(fmat_a, fmat_b, num_modes)

            residual = np.linalg.norm(tensor.data-t_recon.data)
            for i in range(num_modes):
                residual += np.linalg.norm(mlst[i].data-m_recon[i].data)
            self.cost.append(abs(residual)/norm)

            if self.verbose:
                print('Iter {}: relative error of approximation = {}'.format(n_iter, self.cost[-1]))

            # Check termination conditions
            if self.cost[-1] <= self.epsilon:
                if self.verbose:
                    print('Relative error of approximation has reached the acceptable level: {}'
                          .format(self.cost[-1]))
                break
            if self.converged:
                if self.verbose:
                    print('Converged in {} iteration(s)'.format(len(self.cost)))
                break

        if self.verbose and not self.converged and self.cost[-1] > self.epsilon:
            print('Maximum number of iterations ({}) has been reached. '
                  'Variation = {}'.format(self.max_iter, abs(self.cost[-2] - self.cost[-1])))

        # TODO: possibly make another structure
        return fmat_a, fmat_b, t_recon, m_recon

    @property
    def converged(self):
        """ Checks convergence of the CPD-ALS algorithm.
        Returns
        -------
        bool
        """
        # This insures that the cost has been computed at least twice without checking iterations
        try:
            is_converged = abs(self.cost[-2] - self.cost[-1]) <= self.tol
        except IndexError:
            is_converged = False
        return is_converged

    def _init_fmat(self, shape_i, shape_j, rank):
        """ Initialisation of matrices used in CMTF
        Parameters
        ----------
        shape_i : np.ndarray(int)
            Shape[0] of all matrices
        shape_j : np.ndarray(int)
            Shape[1] of all matrices
        rank : int
            The rank specified for factorisation
        Returns
        -------
        (fmat_a, fmat_b) : List(np.ndarray)
            Two lists of the factor matrices
        """
        self.cost = []  # Reset cost every time when method decompose is called
        _r = rank
        
        
        if (np.array(shape_i) < _r).sum() != 0:
            warnings.warn(
                "Specified rank is greater then one of the dimensions of a tensor ({} > {}).\n"
                "Factor matrices have been initialized randomly.".format(_r, shape_i), RuntimeWarning
            )
        fmat_a = [np.random.randn(i_n, _r) for i_n in shape_i]
        fmat_b = [np.random.randn(j_n, _r) for j_n in shape_j]
        return fmat_a, fmat_b

    @staticmethod
    def _reconstruct(fmat_a, fmat_b, n_mat):
        """ Reconstruct the tensor and matrix after the coupled factorisation
        Parameters
        ----------
        fmat_a : List(np.ndarray)
            Multidimensional data obtained from the factorisation
        fmat_b : List(np.ndarray)
            Multidimensional data obtained from the factorisation
        n_mat : int
            Number of matrices provided to fuse
        Returns
        -------
        (core_tensor, lrecon) : np.ndarray or List(np.ndarray)
            Reconstructed tensor and list of matrices obtained from the factorisation
        """
        core_values = np.repeat(np.array([1]), fmat_a[0].shape[1])
        _r = (fmat_a[0].shape[1], )
        core_shape = _r * len(fmat_a)
        core_tensor = super_diag_tensor(core_shape, values=core_values)
        for mode, fmat in enumerate(fmat_a):
            core_tensor.mode_n_product(fmat, mode=mode, inplace=True)
        lrecon = [Tensor(fmat_a[i].dot(fmat_b[i].T)) for i in range(n_mat)]
        return core_tensor, lrecon

    def plot(self):
        print('At the moment, `plot()` is not implemented for the {}'.format(self.name))